In [38]:
import pandas as pd

In [39]:
# file_path = r"G:/다른 컴퓨터/내 노트북/mini_project/서울시 안전표지 관련 정보.csv"/
file_path = "C:/Users/yeeun/Desktop/mini_project/서울시 안전표지 관련 정보.csv"
df = pd.read_csv(file_path, encoding='euc-kr')

C:\Users\yeeun\AppData\Local\Temp\ipykernel_9592\447633685.py:3: DtypeWarning: Columns (0,1,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='euc-kr')


In [40]:
df.head()

,설치방식,상태 (공통),표지코드,비고,X좌표,Y좌표,설치일,교체일,표지관리번호,지주관리번호,각도 (공통),작업구분 (공통),표출구분 (공통),신규정규화ID,이력ID,공사관리번호,표지관리번호.1,표지인덱스,위치정보,공사형태
0,003,001,002,,197430.82666,551261.04855,20130503,20130503,04-0000292950,02-0000145469,0,1,2,3259401,313963,2013-0208-003,04-292950,322,,001
1,999,001,003,T형설치,198212.57833,553261.42560,,,04-0000294826,02-0000178517,0,6,2,3373001,315839,2013-0208-001,04-294826,428,,001
2,003,001,,,200019.71305,554940.85846,,,04-0000296447,02-0000149786,0,1,1,4306633,317460,,04-296447,324,,001
3,003,001,003,,199996.21210,554853.25479,20141109,20141109,04-0000129832,02-0000074717,0,1,2,,99842,2014-0208-031,04-129832,751,9,001
4,003,001,002,,199994.21744,554963.77265,20141109,20141109,04-0000296302,02-0000149781,180,6,2,4306641,317315,2014-0208-031,04-296302,219,,001


In [41]:
df_safety_sign = df[['비고', 'X좌표', 'Y좌표']]
df_safety_sign

,비고,X좌표,Y좌표
0,,197430.82666,551261.04855
1,T형설치,198212.57833,553261.42560
2,,200019.71305,554940.85846
3,,199996.21210,554853.25479
4,,199994.21744,554963.77265
...,...,...,...
349995,,200206.09163,549129.25680
349996,,199043.00000,551338.78125
349997,,198998.59314,551529.46674
349998,,198959.17804,551573.54977


In [42]:
df_safety_sign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   비고      350000 non-null  object 
 1   X좌표     350000 non-null  float64
 2   Y좌표     350000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 8.0+ MB


복지시설 300m 반경 개수 세기

In [43]:
import geopandas as gpd
from shapely.geometry import Point


In [44]:
csv_file_path = r"C:/Users/yeeun/Desktop/mini_project/geocode_welfare_transformed.csv"
df_welfare = pd.read_csv(csv_file_path)

In [45]:
# GeoDataFrame으로 변환
gdf_welfare = gpd.GeoDataFrame(df_welfare, geometry=gpd.points_from_xy(df_welfare['X'], df_welfare['Y']))
gdf_safety_sign = gpd.GeoDataFrame(df_safety_sign, geometry=gpd.points_from_xy(df_safety_sign['X좌표'], df_safety_sign['Y좌표']))

# 결측값이 있는 행 제거 및 인덱스 재설정
gdf_welfare = gdf_welfare[~gdf_welfare[['X', 'Y']].isna().sum(axis=1).astype(bool)].reset_index(drop=True)
gdf_safety_sign = gdf_safety_sign[~gdf_safety_sign[['X좌표', 'Y좌표']].isna().sum(axis=1).astype(bool)].reset_index(drop=True)

In [46]:
buffer_radius = 300  # 반경 300m

# 장애인복지시설의 버퍼 생성
gdf_welfare['buffer'] = gdf_welfare.geometry.buffer(buffer_radius)

# 각 장애인복지시설의 버퍼에 포함된 안전표지판 개수 계산
def count_safety_signs(row):
    try:
        return gdf_safety_sign[row['buffer'].contains(gdf_safety_sign.unary_union)].shape[0]
    except KeyError:
        return 0

gdf_welfare['safety_sign_count'] = gdf_welfare.apply(count_safety_signs, axis=1)

# 필요한 열만 선택
df_safety_sign_count = gdf_welfare[['시설명', 'safety_sign_count']]

KeyboardInterrupt: 

In [ ]:
df_safety_sign_count